In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load ../../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

# Add the uoicorr directory to the path
sys.path.append('../../../uoicorr_run')

# Add the root directory of this repository
sys.path.append('../..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

In addition to the standard set of designs we have been considering, we must consider settings in which the (a) irrepresntible condition is violated with degrees of varying strength (generate random elements from the Wishart distribution)
(b) Network structures of varying topologies
(c) Inverse covariance matrix follows a Wishart distribution: use this to vary the eigenvalue bound on the covariance matrices 
(d) When interpolating between things, look to evenly populate the eigenvalue bound and the irrepresentible strength

In [7]:
def bound_eigenvalue(matrix, k):
    
    # Sort each row
    ordering = np.argsort(matrix, axis = 1)

    # Change to descending order
    ordering = np.fliplr(ordering)
    
    matrix = matrix[ordering]
    # Find the diagonal and move it first
    
    diagonal_locs = np.array([np.where(ordering[i, :] == i)[0][0] 
                              for i in range(ordering.shape[0])])
    for (row, column) in zip(range(ordering.shape[0]), diagonal_locs):
        matrix[row][:column+1] = np.roll(matrix[row][:column+1], 1)
            
    # Sum the first k elements
    row_sums = np.sum(matrix[:, 0:k], axis = 1)

    # Evaluate all Bauer Cassini ovals
    pairs = set(list(itertools.combinations(np.arange(p), 2)))
    ovals = np.array([np.product(row_sums[idx]) for idx in pairs])
    
    # Take the max. This is a bound for any conceivable eigenvalue
    eig_bound = np.max(ovals)
    
    return eig_bound

#### Addressing part (a) by sampling from a Wishart distribution

In [4]:
from scipy.stats import wishart, invwishart

In [6]:
# Generate 1000 random matrices from Wishart_p(n). Keep p fixed at 500. Vary n over a few orders of magnitude
# to see whether this changes the fraction of matrices that violate irrepresentibility

# Vary the sparsity of the model. Assume all beta are positive. Calculate the irrepresentible constant

n = [500, 1000, 5000, 10000]
p = 500
k = np.linspace(10, 500, 10)

n_matrices = 1000

irrep_const = np.zeros((len(n), len(k), n_matrices))
eig_bound = np.zeros(irrep_const.shape)

for i1, n_ in enumerate(n):
    
    for i3, mat in enumerate(range(n_matrices)):
    
        for i2, k_ in enumerate(k):
        
            # Sample from Wishart distribution
            matrix = wishart.rvs(n_, np.eye(p))
                
            # Define the submatrices C11 and C21
            C11 = matrix[np.ix_(np.arange(k_), np.arange(k_))]
            C21 = matrix[np.ix_(np.arange(k_, p), np.arange(k_))]
                
            # Calculate the irrepresentible constant for a fixed subset
            irrep_const[i1, i2, i3] = np.max(C21 @ np.linalg.inv(C11))
            
            # Calculate the subset eigenvalue bound
            eig_bound[i1, i2, i3] = np.power(eig_bound(matrix, k_), -1)

(100, 100)